# 6. Gridsearch

Loading the data

In [81]:
import pandas as pd
import numpy as np

In [82]:
transformed_data = pd.read_csv(r"Datasets\Celebrity Dataset\Celebrity_dataset_transformed.csv")

print(transformed_data.shape)
transformed_data

(450, 65)


,DESPC,DESSC,DESWC,DESPL,DESPLd,DESPLw,DESSL,DESSLd,DESWLsy,DESWLsyd,...,WORD_PROPERTY_AOA_MAX,WORD_PROPERTY_CONCRETENESS,WORD_PROPERTY_PREVALENCE,WORD_SET_INCIDENCE_C4_COMMON_WORDS,Polarity,Overall_Sentiment,Positive_Sentiment,Negative_Sentiment,Neutral_Sentiment,Subjectivity
0,0.444444,0.000000,0.146154,0.027027,0.000000,0.151143,0.682139,0.000000,1.000000,0.563762,...,0.396186,0.393986,0.674648,0.685198,Fake,0.7783,0.033,0.109,0.858,0.332143
1,0.222222,0.176471,0.253846,0.148649,0.176471,0.201948,0.249849,0.248429,0.109532,0.116088,...,0.309322,0.612986,0.682585,0.252618,Fake,0.7763,0.045,0.106,0.849,0.513112
2,0.222222,0.235294,0.276923,0.189189,0.235294,0.220999,0.191726,0.215594,0.199330,0.134906,...,0.297608,0.685996,0.654967,0.546360,Fake,-0.2944,0.150,0.123,0.726,0.397129
3,0.222222,0.176471,0.307692,0.148649,0.176471,0.240051,0.300706,0.189370,0.191279,0.119235,...,0.147246,0.814876,0.426085,0.367094,Fake,0.6486,0.000,0.032,0.968,0.497222
4,0.222222,0.294118,0.296154,0.229730,0.294118,0.220999,0.147614,0.110956,0.139367,0.084885,...,0.617585,0.218604,0.515320,0.550734,Fake,0.9531,0.000,0.145,0.855,0.454167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,0.222222,0.176471,0.296154,0.148649,0.176471,0.227350,0.289808,0.164698,0.264674,0.203276,...,0.306847,0.621988,0.530017,0.516772,True,0.6705,0.017,0.069,0.914,0.428052
446,0.888889,0.117647,0.273077,0.010811,0.037203,0.053345,0.386983,0.384460,0.050249,0.047174,...,0.323093,0.499129,0.691053,0.339440,True,0.9590,0.024,0.180,0.796,0.395543
447,0.444444,0.000000,0.130769,0.000000,0.048029,0.075642,0.645812,0.000000,0.370429,0.251662,...,0.606992,0.835890,0.640981,0.230619,True,0.7964,0.000,0.090,0.910,0.620455
448,0.222222,0.470588,0.534615,0.351351,0.352941,0.337426,0.153582,0.189306,0.225581,0.206593,...,0.423729,0.532790,0.651265,0.156495,True,0.9532,0.089,0.175,0.736,0.473939


In [83]:
transformed_data["Polarity"]

0      Fake
1      Fake
2      Fake
3      Fake
4      Fake
       ... 
445    True
446    True
447    True
448    True
449    True
Name: Polarity, Length: 450, dtype: object

## Gridsearch Fine-tuning


In [84]:
# importing the feature sets
import json

with open( r"Feature_sets\features.json", "r") as f:
    feature_sets = json.load(f)

In [85]:
rfe = feature_sets["rfecv"]

Making the parameter grids we have to take into consideration that the bigger the forest, the easier it will be to overfit.

In [86]:
import numpy as np

n_estimators = [int(x) for x in np.arange(50, 1000, 50)] 
max_def = [None, 2, 3, 4, 6, 8, 10, 12, 15]
criterion = ["gini", "entropy", "log_loss"]

param_grid_rf = {"n_estimators": n_estimators,
                 "max_depth": max_def,
                 "criterion": criterion}

In [87]:
from sklearn.model_selection import GridSearchCV


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


Given that we are going to have a lot of models that may be close in f1 score, I am also going to look at the mean squared error to have a bit more context about the models, which will help with their analysis later.

In [88]:
def gridsearch(data, feature_set, clf, param_grid, cross_val_method, short_name):
    
    X = data[feature_set]
    y = data["Polarity"].replace({"True":1, "Fake":0})
    
    grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=["f1", "roc_auc"], cv=cross_val_method, refit="roc_auc")
    

    grid_search.fit(X, y) # no need to split the data since we are already doing cross validation
                          # we could split X and y to get a validation set however                     
    
    
    results = {}
    results["model"] = grid_search
    results["clf"] = clf
    results["short_name"] = short_name
    results["features"] = feature_set
    results["feature_extraction"] = "rfe"
    results["cross_val_f1"]  = grid_search.best_score_
    results["score_method"] = grid_search.scorer_
    results["best_params"] = grid_search.best_params_
    results["grid_cv_results"] = grid_search.cv_results_
    

    # print(results)
    
    return results, grid_search
    

In [89]:
# result, model = gridsearch(transformed_data, rfe, rf, param_grid_rf, kfold, str(rf))

In [90]:
# import pickle

# pickle.dump(model, open(r"GridSearch\grid_search_rf_rfe.sav", 'wb'))

## Results

In [91]:
# result.keys()

In [92]:
# result["cross_val_f1"]

In [93]:
# result["best_params"]

In [94]:
# result["grid_cv_results"].keys()

In [95]:
# result["grid_cv_results"]["mean_test_f1"]

In [96]:
# result["grid_cv_results"]["mean_test_neg_mean_squared_error"]

In [97]:
# grids = result["grid_cv_results"]
# df_grids = pd.DataFrame(grids)
# df_grids.columns

In [98]:
# # saving the results

# df_grids.to_csv(r"GridSearch\grid_cv_results.csv")
